In [8]:
from keras.models import Sequential
from keras.layers import Dropout, LSTM, Activation, Dense
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from pickle import dump

/Users/cmarquis/Documents/penn/penn-ml-challenge/venv/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/cmarquis/Documents/penn/penn-ml-challenge/venv/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/cmarquis/Documents/penn/penn-ml-challenge/venv/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.d

In [9]:
data = pd.read_csv('bitcoin.csv').drop(['time_period_start', 'time_period_end', 'time_open', 'time_close'], axis=1)
data

,price_open,price_high,price_low,price_close,volume_traded,trades_count
0,7165.72,7165.72,7165.71,7165.71,0.021841,2
1,7168.30,7168.30,7168.30,7168.30,1.000000,2
2,7170.50,7170.50,7170.50,7170.50,0.002000,1
3,7169.20,7169.20,7169.20,7169.20,0.004000,2
4,7169.20,7169.20,7169.20,7169.20,0.002000,1
...,...,...,...,...,...,...
9995,7179.50,7179.50,7179.50,7179.50,0.013325,1
9996,7181.67,7181.67,7181.67,7181.67,0.013364,1
9997,7179.50,7179.50,7179.50,7179.50,0.001526,1
9998,7182.12,7182.12,7182.12,7182.12,0.013437,1


## Lookback Window

In order to capture the past as features, we create the below function. It takes as input some number of seconds to look back, defaulted to 60 seconds.

Question: Is this a good feature set? Can you find a better one?

In [11]:
def lookback(dataset, timesteps = 60):
    # this uses the shift method of pandas dataframes to shift all of the columns down one row
    # and then append to the original dataset
    data = dataset
    for i in range(1, timesteps):
        step_back = dataset.shift(i).reset_index()
        step_back.columns = ['index'] + [f'{column}_-{i}' for column in dataset.columns if column != 'index']
        data = data.reset_index().merge(step_back, on='index', ).drop('index', axis=1)
        
    return data.dropna()
        
features = lookback(data)
features

,price_open,price_high,price_low,price_close,volume_traded,trades_count,price_open_-1,price_high_-1,price_low_-1,price_close_-1,...,price_low_-58,price_close_-58,volume_traded_-58,trades_count_-58,price_open_-59,price_high_-59,price_low_-59,price_close_-59,volume_traded_-59,trades_count_-59
59,7154.97,7154.97,7154.97,7154.97,2.000000,2,7163.30,7163.30,7163.30,7163.30,...,7168.30,7168.30,1.000000,2.0,7165.72,7165.72,7165.71,7165.71,0.021841,2.0
60,7161.20,7163.40,7161.20,7163.40,0.015800,2,7154.97,7154.97,7154.97,7154.97,...,7170.50,7170.50,0.002000,1.0,7168.30,7168.30,7168.30,7168.30,1.000000,2.0
61,7154.98,7154.98,7154.97,7154.98,0.038357,3,7161.20,7163.40,7161.20,7163.40,...,7169.20,7169.20,0.004000,2.0,7170.50,7170.50,7170.50,7170.50,0.002000,1.0
62,7154.98,7154.98,7154.98,7154.98,0.032201,1,7154.98,7154.98,7154.97,7154.98,...,7169.20,7169.20,0.002000,1.0,7169.20,7169.20,7169.20,7169.20,0.004000,2.0
63,7154.97,7154.97,7154.97,7154.97,2.000000,1,7154.98,7154.98,7154.98,7154.98,...,7165.72,7169.20,0.075433,3.0,7169.20,7169.20,7169.20,7169.20,0.002000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,7179.50,7179.50,7179.50,7179.50,0.013325,1,7179.04,7179.04,7178.23,7178.23,...,7178.64,7178.64,0.006793,1.0,7179.88,7179.88,7179.88,7179.88,0.009628,1.0
9996,7181.67,7181.67,7181.67,7181.67,0.013364,1,7179.50,7179.50,7179.50,7179.50,...,7178.64,7178.64,0.001249,1.0,7178.64,7178.64,7178.64,7178.64,0.006793,1.0
9997,7179.50,7179.50,7179.50,7179.50,0.001526,1,7181.67,7181.67,7181.67,7181.67,...,7178.64,7178.64,0.068287,1.0,7178.64,7178.64,7178.64,7178.64,0.001249,1.0
9998,7182.12,7182.12,7182.12,7182.12,0.013437,1,7179.50,7179.50,7179.50,7179.50,...,7173.77,7173.77,0.008064,1.0,7178.64,7178.64,7178.64,7178.64,0.068287,1.0


In [10]:
# Use min max scaler to transform feature data 
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(data.values)
scaled_df = pd.DataFrame(scaled_values, index = data.index, columns = data.columns)
scaled_df

,price_open,price_high,price_low,price_close,volume_traded,trades_count
0,0.267495,0.264047,0.339554,0.267368,3.358218e-04,0.011494
1,0.300203,0.296333,0.369204,0.300203,1.538114e-02,0.011494
2,0.328093,0.323864,0.394390,0.328093,3.063947e-05,0.000000
3,0.311613,0.307596,0.379508,0.311613,6.140199e-05,0.011494
4,0.311613,0.307596,0.379508,0.311613,3.063947e-05,0.000000
...,...,...,...,...,...,...
9995,0.442191,0.436491,0.497424,0.442191,2.048259e-04,0.000000
9996,0.469701,0.463647,0.522267,0.469701,2.054366e-04,0.000000
9997,0.442191,0.436491,0.497424,0.442191,2.334721e-05,0.000000
9998,0.475406,0.469278,0.527418,0.475406,2.065548e-04,0.000000


In [11]:
# save scaler
dump(scaler, open('scaler.pkl', 'wb'))

In [12]:
# define dimensions fo an input series in 
num_in  = 59
num_out = 1
n_features = scaled_df.values.shape[1]

# Splitting the data into appropriate sequences
X, y = [], []
    
for i in range(len(scaled_df.values)):
    end = i + num_in
    out_end = end + num_out

    if out_end > len(scaled_df.values):
        break

    seq_x, seq_y = scaled_df.values[i:end][:], scaled_df['price_high'].values[end:out_end]

    X.append(seq_x)
    y.append(seq_y)


X=np.array(X)
print(X.shape)
y=np.array(y)
print(y.shape)

# Reshaping the X variable from 2D to 3D
X = X.reshape((X.shape[0], X.shape[1], n_features))

(9941, 59, 6)
(9941, 1)


In [29]:
# Use rnn lstm model 
model = Sequential()
model.add(LSTM(30, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1, activation='relu'))

model.compile(
    loss='mse',
    optimizer=Adam(lr=0.01), # is this the best optimizer/learning rate?
    metrics=['mean_squared_error', 'mean_absolute_error'] # does accuracy make sense in this context?
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    mode='auto',
    restore_best_weights=True
)

res = model.fit(X, y, epochs=20, batch_size=30, validation_split=0.1)


Epoch 1/20
299/299 [==============================] - 5s 16ms/step - loss: 0.0024 - mean_squared_error: 0.0024 - mean_absolute_error: 0.0241 - val_loss: 0.0014 - val_mean_squared_error: 0.0014 - val_mean_absolute_error: 0.0328
Epoch 2/20
299/299 [==============================] - 5s 16ms/step - loss: 7.3270e-04 - mean_squared_error: 7.3270e-04 - mean_absolute_error: 0.0193 - val_loss: 7.3788e-04 - val_mean_squared_error: 7.3788e-04 - val_mean_absolute_error: 0.0201
Epoch 3/20
299/299 [==============================] - 5s 16ms/step - loss: 6.9198e-04 - mean_squared_error: 6.9198e-04 - mean_absolute_error: 0.0186 - val_loss: 9.0629e-04 - val_mean_squared_error: 9.0629e-04 - val_mean_absolute_error: 0.0238
Epoch 4/20
299/299 [==============================] - 5s 16ms/step - loss: 6.8190e-04 - mean_squared_error: 6.8190e-04 - mean_absolute_error: 0.0183 - val_loss: 8.2536e-04 - val_mean_squared_error: 8.2536e-04 - val_mean_absolute_error: 0.0222
Epoch 5/20
299/299 [========================

In [11]:
# model = Sequential()
# model.add(Dense(32, input_dim=features.shape[1]))
# model.add(Dense(16))
# model.add(Dense(1, activation='relu'))

# model.compile(
#     loss='mse',
#     optimizer=Adam(lr=0.01), # is this the best optimizer/learning rate?
#     metrics=['mean_squared_error', 'mean_absolute_error'] # does accuracy make sense in this context?
# )

# ## callbacks
# early_stopping = EarlyStopping(
#     monitor='val_loss',
#     mode='auto',
#     restore_best_weights=True
# )


# history = model.fit(
#     features,
#     target,
#     validation_split=.3,
#     epochs=20,
#     verbose=1
# )

Epoch 1/20
218/218 [==============================] - 0s 2ms/step - loss: 51482672.0000 - mean_squared_error: 51482672.0000 - mean_absolute_error: 7170.7412 - val_loss: 51821536.0000 - val_mean_squared_error: 51821536.0000 - val_mean_absolute_error: 7198.7134
Epoch 2/20
218/218 [==============================] - 0s 962us/step - loss: 51664692.0000 - mean_squared_error: 51664692.0000 - mean_absolute_error: 7187.7871 - val_loss: 51821536.0000 - val_mean_squared_error: 51821536.0000 - val_mean_absolute_error: 7198.7134
Epoch 3/20
218/218 [==============================] - 0s 1ms/step - loss: 51664704.0000 - mean_squared_error: 51664704.0000 - mean_absolute_error: 7187.7881 - val_loss: 51821536.0000 - val_mean_squared_error: 51821536.0000 - val_mean_absolute_error: 7198.7134
Epoch 4/20
218/218 [==============================] - 0s 1ms/step - loss: 51664696.0000 - mean_squared_error: 51664696.0000 - mean_absolute_error: 7187.7891 - val_loss: 51821536.0000 - val_mean_squared_error: 51821536.

In [13]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 60, 30)            4440      
_________________________________________________________________
dense_3 (Dense)              (None, 60, 1)             31        
Total params: 4,471
Trainable params: 4,471
Non-trainable params: 0
_________________________________________________________________


Below we calculate predictions and root mean square error. Can we easily improve this RMSE?

In [14]:
## prediction
temp = np.array(X)
temp.shape
predictions = model.predict(temp)
predictions

for_transform = [[0, predictions[0][0], 0, 0, 0, 0]]

res = scaler.inverse_transform(for_transform)
res[0][1]

7159.6842652085425

In [28]:
target = y
rmse = np.sqrt(np.mean(np.square((target.reshape(-1, 1) - predictions))))
rmse

0.02548257702982088

## Save Model

The last thing we'll do is save the model for use in the API.

In [73]:
model.save('model.h5')